# 6. Tuning

In [1]:
MODEL = 'RFR'
TUNING_MODEL = 'guanganb'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
import os
import pickle
py_file_location = "./drive/My Drive/PrivatePackages"
sys.path.append(os.path.abspath(py_file_location))

py_file_location = "./drive/My Drive/LAB/BrownlowH2H/PrivatePackages"
sys.path.append(os.path.abspath(py_file_location))

import NingXiang

import pandas as pd

In [4]:
ningxiang = NingXiang.NingXiang()

NingXiang Initialised


Read in and Prepare Data

In [5]:
train_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/train_data.parquet')
val_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/val_data.parquet')
test_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/test_data.parquet')

In [6]:
train_other_info = train_data[['game_id', 'player1', 'player2']]
val_other_info = val_data[['game_id', 'player1', 'player2']]
test_other_info = test_data[['game_id', 'player1', 'player2']]

In [7]:
train_x = train_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
train_y = train_data['Brownlow Votes']
val_x = val_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
val_y = val_data['Brownlow Votes']
test_x = test_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
test_y = test_data['Brownlow Votes']

In [8]:
# ningxiang.read_in_train_data(train_x, train_y)
# ningxiang.set_model_type("Regression")
# ningxiang_output = ningxiang.get_rf_based_feature_combinations(3)

# with open(f'./drive/My Drive/LAB/BrownlowH2H/models/NingXing_output.pickle', 'rb') as f:
#     ningxiang_output = pickle.load(f)

In [9]:
with open('./drive/My Drive/LAB/BrownlowH2H/models/H2H_NingXiang.pickle', 'rb') as f:
  ningxiang_output = pickle.load(f)

Set parameters

In [10]:
import GuangAnB_H2H

In [11]:
guangan = GuangAnB_H2H.GuangAnB()

GuangAn Initialised


In [12]:
from sklearn.ensemble import RandomForestRegressor as RFR

In [13]:
guangan.read_in_model(RFR, 'Regression')

Successfully read in model <class 'sklearn.ensemble._forest.RandomForestRegressor'>, which is a Regression model


In [14]:
guangan.read_in_data(train_x, train_y, val_x, val_y, test_x, test_y)

Read in Train X data
Read in Train y data
Read in Val X data
Read in Val y data
Read in Test X data
Read in Test y data


In [15]:
parameter_choices = {
    'n_estimators': set(range(25, 401)),
    'max_depth': set(range(3, 193)),
    'max_samples': (0.4, 0.85),
    'max_features': (0.4, 0.85),
    'ccp_alpha': (-8, -1)
}

In [16]:
guangan.set_hyperparameters(parameter_choices)

Successfully recorded hyperparameter choices


In [17]:
guangan.read_in_transform({'ccp_alpha': '10^'})

Updated transform dictionary: {'n_estimators': False, 'max_depth': False, 'max_samples': False, 'max_features': False, 'ccp_alpha': '10^'}


In [18]:
guangan.set_non_tuneable_hyperparameters({'random_state': 19260817, 'n_jobs': -1})

Successfully recorded non_tuneable_hyperparameter choices


In [19]:
guangan.set_features(ningxiang_output)

Successfully recorded tuneable feature combination choices and updated relevant internal structures


In [20]:
try:
  guangan.read_in_tuning_result_df(f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.csv', f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.pickle')
  print('True')
except:
  print('False')

Successfully read in tuning result of 2249 rows
True


In [21]:
guangan.set_tuning_result_saving_address(f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.csv')

Successfully set tuning output address


In [22]:
guangan.set_best_model_saving_address(f'./drive/My Drive/LAB/BrownlowH2H/models/tmp_models/{TUNING_MODEL}_BrownlowH2H_{MODEL}.pickle')

Successfully set best model output address


In [23]:
guangan.train_other_info = train_other_info
guangan.val_other_info = val_other_info
guangan.test_other_info = test_other_info

In [24]:
import warnings
warnings.filterwarnings("ignore")

In [25]:
guangan.tune()

Streaming output truncated to the last 5000 lines.
        Has trained 2249 combinations so far
Already Trained and Tested combination {'n_estimators': 166, 'max_depth': 27, 'max_samples': 0.79375, 'max_features': 0.68125, 'ccp_alpha': -5.375, 'features': 35}, which had val score of 0.6111
        Current best combo: (119, 98, 0.85, 0.7375, -8.0, 35) with val score 0.6488. 
        Has trained 2249 combinations so far
Already Trained and Tested combination {'n_estimators': 166, 'max_depth': 27, 'max_samples': 0.79375, 'max_features': 0.68125, 'ccp_alpha': -4.5, 'features': 31}, which had val score of 0.6132
        Current best combo: (119, 98, 0.85, 0.7375, -8.0, 35) with val score 0.6488. 
        Has trained 2249 combinations so far
Already Trained and Tested combination {'n_estimators': 166, 'max_depth': 27, 'max_samples': 0.79375, 'max_features': 0.68125, 'ccp_alpha': -4.5, 'features': 35}, which had val score of 0.5975
        Current best combo: (119, 98, 0.85, 0.7375, -8.0, 35)

In [26]:
guangan.view_best_combo_and_score()

Max Score: 
 0.6488469601677149
Max Test Score: 
 0.648847
Max Combo Index: 
 (119, 98, 0.85, 0.7375, -8.0, 35)
Max Combo Hyperparamer Combination: 
 {'n_estimators': 119, 'max_depth': 98, 'max_samples': 0.85, 'max_features': 0.7375, 'ccp_alpha': -8.0, 'features': 35}
Max Combo Features: 
 ('AFLCA_votes', 'Disposals', 'Contested Possessions', 'Goals', 'Centre Clearances', 'Intercepts', 'Clearances', 'Marks Inside 50', 'Hitouts', 'Handballs', 'Behinds', 'Contested Marks', 'Bounces', 'Ineffective Disposals', 'Stoppage Clearances', 'Tackles Inside 50', 'Rebound 50s', 'One Percenters', 'Goal Assists', 'Uncontested Possessions', 'Frees Agains', 'Inside 50s', 'Frees For', 'Metres Gained', 'Score Involvements', 'Behind Assists', 'Tackles Outside 50', 'Tackles', 'Clangers', 'Uncontested Marks', 'Marks Outside 50', 'Kicks', 'Turnovers', 'Marks', 'Time On Ground %', 'Effective Disposals', 'HomeAway', 'Winloss')
# Combos Checked: 2249
